In [239]:
import pandas as pd
import pickle
import csv
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

from collections import defaultdict
from collections import Counter
from math import log
import numpy as np

from scipy.spatial.distance import cosine

import math 

import heapq
from heapq import heappush, heappop

#from geopy.geocoders import Nominatim
#from geopy import distance

sp = string.punctuation+'“”–’'

[nltk_data] Downloading package punkt to /Users/miguel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/miguel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import time

# Import data

In [8]:
texas1 = pd.read_csv("data/Airbnb_Texas_Rentals.csv")
texas1 = texas1.drop(['Unnamed: 0'], axis=1)

In [9]:
#pd.set_option('expand_frame_repr',False)
texas1.head()

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...
1,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...
2,$59,1,Houston,January 2017,'River house on island close to the city' \nA ...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...
3,$60,1,Bryan,February 2016,Private bedroom in a cute little home situated...,30.637304,-96.337846,Private Room Close to Campus,https://www.airbnb.com/rooms/11839729?location...
4,$75,2,Fort Worth,February 2017,Welcome to our original 1920's home. We recent...,32.747097,-97.286434,The Porch,https://www.airbnb.com/rooms/17325114?location...


# Create docuemnts as tsv file

In [3]:
path1 = 'data/docs/'
path2 = '.tsv'
stemmed_path='data/tokenized_docs/'
sp = string.punctuation+'“”–’'

In [171]:
def remove_step(doc):
    """
    takes as input the string of the document
    removes stopwords, punctuation and makes stemming 
    input:
    - string of document
    output:
    - list of term after stemming process
    
    """
    
    # check if it's a nan value 

    if isinstance(doc, float):
        return str(doc)
    
    doc=doc.replace("\\n", " ")
    # punctuations
    doc = [ c if c not in sp else " " for c in doc ]
    doc = ''.join(doc)
    # stopwords
    doc = [ word for word in doc.split() if word.lower() not in stopwords.words('english') ]
    doc = ' '.join(doc)
    
    # stemming
    ps = PorterStemmer()
    words = word_tokenize(doc)
    
    w_lst = []
    for w in words:
        w_lst.append(ps.stem(w))
    
    # something else
    
    return ' '.join(w_lst)

In [5]:
def save_obj(obj, name):
    """
    save the object int a pickle file
    """
    with open('data/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    """
    load the object from a pickle file
    """
    with open('data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

Preprocessing create the tsv files.
We created vocabulary parsing every document and updating it when the algorithm finds a new word. 
Format of vocabulary => 'string':integer.
Creation of ii1:
The first inverted index is built, updating it every time a word is find in a document.
At the end we stored vocabulary and the inverted index using pickle.

In [187]:
def preprocessing(data):
    n=len(data)

    for i in range(n):
        with open(path1 + 'doc_'+ str(i) + '.tsv', 'w', encoding = "utf-8") as doc:
            a = csv.writer(doc, delimiter='\t')
            a.writerow([data.iloc[i]['average_rate_per_night'],data.iloc[i]['bedrooms_count'], data.iloc[i]['city'],
                        data.iloc[i]['date_of_listing'], data.iloc[i]['description'],
                        data.iloc[i]['latitude'],
                        data.iloc[i]['longitude'],
                        data.iloc[i]['title'],
                        data.iloc[i]['url']])

    return

def create_vocabulary_and_ii1 (data):
    n = len(data)
    vocabulary = {}
    ii1 = {}    
    cnt = 0
    
    for i in range(n):
        # creating a tokenized string with title and description
        tokenized_str = (remove_step(data.iloc[i]['title']) + ' ' 
                                     + remove_step(data.iloc[i]['description']))

        # creating the dictionary
        for term in tokenized_str.split(' '):
            if term in vocabulary.keys():
                term_id = vocabulary[term]
            else:
                vocabulary[term] = cnt
                term_id = cnt
                cnt+=1

            if term_id not in ii1.keys():
                ii1[term_id] = ['doc_'+str(i)]
            else:
                lista = ii1[term_id]
                document = 'doc_'+str(i)
                if document in lista:
                    continue
                else:        
                    ii1[term_id].append('doc_'+str(i))

    # store vocabulary in pickle format
    save_obj(vocabulary, 'vocabulary')
    save_obj(ii1, 'inverted_index_1')
    return
            

In [9]:
#create_vocabulary and inverted index 1 (for the first search engine)
create_vocabulary_and_ii1(texas1)

In [188]:
# do not run if you don't want to waste time :-D 
preprocessing(texas1)

In [176]:
def search_engine_1(query): 
    
    query = remove_step(query)
    query = list(set(query.split(' ')))
    
    lst_of_lst=[]
    
    vocabulary = load_obj('vocabulary')
    ii1 = load_obj('inverted_index_1')
    
    for w in query:
        if w not in vocabulary:
            print('No results')
            return
        i = vocabulary[w]
        lst_of_lst.append(ii1[i])


    doc_list = set.intersection(*[set(sublist) for sublist in lst_of_lst])
    doc_list = list(doc_list)
    dl = len(doc_list)
    
    if dl ==0:
        print('No results')
        return

    list_for_df=[]
    for i in range(dl):
        with open ("data/docs/" + doc_list[i] + '.tsv') as doc:
            row = doc.read()
            lst = row.split('\t')
            lst = [lst[7],lst[4],lst[2],lst[8]]
            list_for_df.append(lst)
        
    df=pd.DataFrame(list_for_df, columns=['Title', 'Description', 'City', 'Url'])
    
    return df.head(5)   
        

In [177]:
vocabulary = load_obj('vocabulary')

In [179]:
search_engine_1('Beautiful room with garden')


,Title,Description,City,Url
0,The Vintage room in Fort Worth,Our place is a beautiful cozy open concept hou...,Fort Worth,https://www.airbnb.com/rooms/18959678?location...
1,The Vintage room in Fort Worth,Our place is a beautiful cozy open concept hou...,Fort Worth,https://www.airbnb.com/rooms/18959678?location...
2,Colorful Garden Home plus Private Studio Apart...,"Natural beauty and great art make this clean, ...",San Antonio,https://www.airbnb.com/rooms/17341454?location...
3,Simple room at centralized location,Private room at centralized location. The rest...,Dallas,https://www.airbnb.com/rooms/4196708?location=...
4,Beautiful Condo. Direct access to the Beach.,Beautiful Condo with direct access the South P...,South Padre Island,https://www.airbnb.com/rooms/17045040?location...


In [27]:
def dict_with_tf(data):
    vocabulary = load_obj('vocabulary')
    n = len(data)
    
    ii2 = defaultdict(list)
    
    
    for i in range(n):
        tokenized_str = (remove_step(data.iloc[i]['title']) + ' ' 
                                     + remove_step(data.iloc[i]['description']))
    
        for term in tokenized_str.split(' '):
            doc_name = 'doc_%s'%i
            ii2[term].append((doc_name,1))
        
    return ii2

    

In [28]:
ii2 = dict_with_tf(texas1)

In [85]:
save_obj(ii2,'inverted_index_onlyTF')

    II with TF:
    Taking each list for each term in the inverted index,
    we wanted to find the occurrencies for that term in each document.
    Thus, we created the inverted index with every document appended inside the value of each 
    term (the key of the dictionary), 
    and using in a loop the reduce_doc_list method.
    In this method we reduced the repetitions of the same docs in each list, summing them. 
   

In [30]:
def reduce_doc_list(doc_list):
    """
    function called by dict_TFIDF
    
    It reduces the list of documents into a list 
    of tuple with doc_id and its occurencies
    
    input:
    - list 
    output:
    - list 
    """
    tf_term_i = Counter(doc_list)
    doc_tf_lst = []
    doc_tf_lst = [tuple([key,value]) for key,value in tf_term_i.items()]
    return doc_tf_lst

def compute_ii2_TFIDF(ii2,n):
    """
    compute the ii2_TFIDF
    input:
    - inverted index matrix (with TF)
    - number of documents
    output:
    - ii2 
    """
    for key, value in ii2.items():
        N = len(value)
        new_list = []
        for item in value:
            new_list.append(tuple([item[0], round(float(item[1])* log(n/N),3)]))
            
        ii2[key] = new_list
    return ii2

In [31]:
def dict_with_TFIDF(data):
    """
    creates the TFIDF inverted index as dict
    and store it into a pickle file
    input:
    - data
    """
    vocabulary = load_obj('vocabulary')
    n = len(data)
    
    ii2 = defaultdict(list)
    
    for i in range(n):
        tokenized_str = (remove_step(data.iloc[i]['title']) + ' ' 
                                     + remove_step(data.iloc[i]['description']))
    
        for term in tokenized_str.split(' '):
            doc_name = 'doc_%s'%i
            ii2[vocabulary[term]].append(doc_name)
            
    
    for key,value in ii2.items():
        ii2[key] = reduce_doc_list(value)
    
    ii2 = compute_ii2_TFIDF(ii2,n)
    save_obj(ii2,'inverted_index_TFIDF')
    return 

In [23]:
start = time.time()
dict_with_TFIDF(texas1)
print (time.time()-start)

242.7152338027954


We decided to create the new search engine (second) basing on the inverted indexes already built, but cutting them: in this way the laptop will have less computation than to compute everything.
Thus, every time we have a new query, the method generates 2 different inverted indexes.

In this way, we won't need a big numpy array for the query and each document considered (we took docs that contained every word, using intersection (for this purpose we took the first inverted index)) for making the cosine similarity (for this we exploited the second inverted index) ; in fact, vectors for computing cosine similarity will be exactly long like the length of the query. 
Query will have 1 for each element (?)(still to decide) and doc vector will have each tfidf for each term of the query.

$$ 1-cosine(\text{doc array for the query},\text{array of query}) $$
will compute the Cosine Similarity


In [231]:
def search_engine_2(query, k = 5):
    """
    search engine with TFIDF
    input:
    - query: the query (string)
    - k: number of output as doc (default = 5)
    output:
    - df: dataframe with k documents
    """

    # stemming the query with remove step method
    query = remove_step(query).split(' ')
    vocabulary = load_obj('vocabulary')
    
    # filtering the values not contained in vocabulary
    # dict and mapping each term to its term_ID value
    query = filter(lambda x: x in vocabulary.keys(),query)
    query = list(map(lambda x: vocabulary[x], query))
    
    # making query list and array at the same time 
    # with Counter() dict (to ensure the order of the 2 arrays)
    # (dictionaries are not ordered)
    query_array = []
    query_list = []
    for key, frequence in Counter(query).items():
        query_list.append(key)
        query_array.append(frequence)
    
    # creating query list and numpy array with frequencies
    query = query_list
    query_array = np.array([query_array])

    # list of documents obtained intersecating each list
    # from inverted index 1 (easier to compute)
    docs = []
    ii1 = load_obj('inverted_index_1')
    ii1 = {term_id:ii1[term_id] for term_id in query}
    docs = set.intersection(*[set(value) for value in ii1.values()])

    # filtering ii2 with only the query terms
    ii2 = load_obj('inverted_index_TFIDF')
    ii2 = {term_id:ii2[term_id] for term_id in query}
    
    # for each doc in the list docs it creates the ranking list
    rank_lst = []
    for doc_id in docs:
        # doc TFIDF array. It must have the same 'order' as
        # the query array. to ensure this we use the query list
        array = []
        for item in query:
            array += tuple(filter(lambda x:x[0]==doc_id, ii2[item]))

        array = np.array([x[1] for x in array])
        rank_lst.append(tuple([1-cosine(array,query_array),doc_id]))
    
    # print('%d documents'%len(rank_lst)) DEBUG
    
    df = first_k_documents(rank_lst, 5)
    
    return len(rank_lst)
    return df


def first_k_documents(heap_rank_lst, k = 5):
    """
    it returns the first k documents as a df
    loading them into an heap memory
    input:
    - heap_rank_lst: list to be heaped
    - k: first k documents
    output:
    - df: dataframe with k documents
    """
    list_for_df = []
    
    if k > len(heap_rank_lst):
        k = len(heap_rank_lst)
        
    heapq._heapify_max(heap_rank_lst)
    
    for i in range(k):
        current_doc = heapq._heappop_max(heap_rank_lst)
    
        with open ("data/docs/" + current_doc[1] + '.tsv') as doc:
            row = doc.read()
            lst = row.split('\t')
            lst = [lst[7],lst[4],lst[2],lst[8], round(current_doc[0],3)]
            list_for_df.append(lst)
    
    return pd.DataFrame(list_for_df, columns=['Title', 'Description', 'City', 'Url', 'Similarity'])



In [234]:
from time import time
start = time()
df = search_engine_2("search engine")
print(time()-start)
df

0.6097888946533203


0

## 4 Point


In [222]:
rooms_ranker = {
    0: 1,
    1: 0.7,
    2: 0.4,
    3: 0.1,
    # others values are almost zero! 
}

def rooms_rank (rooms, nrooms, rooms_ranker):
    """
    ranks the room
    """
    
    # if rooms or nrooms is a string (eg: studio)
    # convert it into a string
    if isinstance(rooms, str):
        rooms = 1
    if isinstance(nrooms, str):
        nrooms = 1
    
    # taking the distance between the two rooms
    value = abs(rooms - nrooms)

    # if there's no a region of values give zero
    if value not in rooms_ranker:
        return 0
    else:
        # give the value signed in rooms_ranker
        return rooms_ranker[value]

def price_rank (p_query, p_doc):
    
    if p_query == p_doc:
        return 1
    
    if p_query > p_doc:
        return p_doc/p_query
    else:
        return p_query/p_doc

In [198]:
# other price raker values! 
# if we don't know how to handle the problem
price_ranker = {
    
    10: 1,
    20: 0.8,
    40: 0.6,
    60: 0.4,
    80: 0.2   
}

A = 10
B = 20 
C = 40 
D = 60
E = 80

def rooms_rank (n_query, n_doc, rooms_ranker):
    value = abs(n_query)


In [224]:
def search_engine_3(query): 
    
    query = remove_step(query)
    query = list(set(query.split(' ')))
    
    heap = []
    lst_of_lst=[]
    
    vocabulary = load_obj('vocabulary')
    ii1 = load_obj('inverted_index_1')
    
    for w in query:
        if w not in vocabulary:
            print('No results')
            return
        i = vocabulary[w]
        lst_of_lst.append(ii1[i])


    doc_list = set.intersection(*[set(sublist) for sublist in lst_of_lst])
    doc_list = list(doc_list)
    dl = len(doc_list)   
    
    if dl == 0:
        print('No results')
        return
    
    print("max price:")
    rmax = int(input())
    print("n rooms:")
    rooms = int(input())
    #if len(str(rmin)) == 0:
    #    rmin = 1
    for doc in doc_list:
        with open('data/docs/'+doc+'.tsv', encoding = 'utf-8') as f:
            row = f.read()
            row = row.split('\t')
            dp = int(row[0][1:])
            try:
                nrooms = int(row[1])
            except ValueError:
                continue
            if math.isnan(float(row[1])):
                continue

             # and dp > rmin: 
            #sp = abs(rmax - rmin)/dp
            score_price = 0.8*price_rank(rmax,dp)
            ratio_rooms = 0.2*rooms_rank(rooms, nrooms, rooms_ranker)
                #if ratio_rooms <= 1 and ratio_rooms > 0.5 and sp > 0.5 and sp <=1:  
            heappush(heap,tuple([score_price+ratio_rooms,doc])) 
    
    list_for_df=[]
    k=5
    
    l = len(heap)
    for i in range(l):
        heapq._heapify_max(heap)
        tup = heappop(heap)
        #print(tup)
        with open ("data/docs/" + tup[1] + '.tsv', encoding = 'utf-8') as doc:
            row = doc.read()
            lst = row.split('\t')
            lst = [i, lst[7],lst[4],lst[2],lst[8]]
            list_for_df.append(lst)
        
    df=pd.DataFrame(list_for_df, columns=['Ranking','Title', 'Description', 'City', 'Url'])
    
    return df.head(k),tup
    

In [227]:
res,tup = search_engine_3("texas room private")

max price:
3
n rooms:
3


In [237]:
def scoring_distance(center, house):
      
    #R = distance.distance(center, house).km
    #print(R)
    R  = 6
    #print(R)
    
    if R <= 1: 
        return 1
    else: 
        #return 1 - R / (1 + R)
        #return 1 - (math.log(R)  + 1) / (1 + (math.log(R) + 1))
        return 1- ((R*1/2 + 1) / (1 + (R*1/2 + 1)))
        #return 3**(-R)# 1/R



In [236]:

geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = input('where : ')
location = geolocator.geocode(location)
center = [location.latitude, location.longitude]

docs = ['doc_2']
for i in range(len(docs)):

    with open ("data/docs/" + str(docs[i]) + '.tsv') as doc:

        row = doc.read()
        row = row.split('\t')
        house = [float(row[5]),float(row[6])]
        
        
        if math.isnan(house[0]):
                # extract the location from the name of the city
                location = geolocator.geocode(row[2])
                house = [location.latitude, location.longitude]

        print(scoring_distance(center, house))



NameError: name 'Nominatim' is not defined